# Data Science Methodology for Breast Cancer Diagnosis (DSM-BCD)

**Problema.** Colombia presenta limitaciones con respecto al acceso de la detección y el diagnóstico temprano del cáncer, provocado en la mayoría de los casos por factores como el estrato socio-económico, la cobertura del seguro de salud, el origen y la accesibilidad. En promedio, el tiempo de espera de un paciente es de 90 días desde la aparición de los síntomas hasta el diagnóstico de dicho cáncer. La primera acción para reducir la tasa de mortalidad por cáncer de mama debe estar enfocada en la agilidad del diagnóstico y el acceso oportuno a la atención.  Así, el objetivo de esta investigación es aplicar las etapas de la metodología KDD\footnote{Knowledge Discovery in Databases} al conjunto de datos de morbilidad por cáncer entre los años 2019 y 2020 en el municipio de Pereira-Risaralda. Esto con la finalidad de pronosticar y caracterizar el tipo de población mas susceptible de padecer esta enfermedad según su edad, genero, zona y régimen de salud.

**Conjunto de datos.** Un total de 817 muestras de tumores de mama se perfilaron con cinco plataformas diferentes como se ha descrito previamente (Cancer Genome Atlas Research Network, 2014) y también se perfilaron 633 casos mediante matriz de proteínas en fase inversa (RPPA)


## Análisis exploratorio de datos

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
from dataprep.eda import plot, plot_correlation, plot_missing,create_report
from dataprep.clean import clean_headers
from dataprep.clean import clean_duplication
import numpy as np

In [ ]:
with open('brca_tcga_pub2015_clinical_data.csv') as f:
    breast_cancer=pd.read_csv(f, delimiter=';')

In [ ]:
breast_cancer.head(5)

In [ ]:
breast_cancer=clean_headers(breast_cancer)

In [ ]:
breast_cancer

In [ ]:
#b=plot(breast_cancer)
#b.show_browser()

In [ ]:
#report = create_report(breast_cancer, title='Breast Invasive Carcinoma (TCGA, Cell 2015)')

In [ ]:
#report.show_browser()

In [ ]:
#report.save()

In [ ]:
# Eliminacion de fechas
# Eliminacion de fechas
breast_clean=breast_cancer.drop(
    ['i_study_id',
    'patient_id',
    'sample_id',
    'american_joint_committee_on_cancer_publication_version_type',
    'brachytherapy_first_reference_point_administered_total_dose',
    'cancer_type',
    'birth_from_initial_pathologic_diagnosis_date',
    'days_to_sample_collection',
    'death_from_initial_pathologic_diagnosis_date',
    'last_alive_less_initial_pathologic_diagnosis_date_calculated_day_value',
    'days_to_last_followup',
    'disease_free_months',
    'disease_code',
    'form_completion_date',
    'international_classification_of_diseases_for_oncology_third_edition_icd_o_3_histology_code',
    'international_classification_of_diseases_for_oncology_third_edition_icd_o_3_site_code',
    'informed_consent_verified',
    'year_cancer_initial_diagnosis',
    'is_ffpe',
    'overall_survival_months',
    'other_patient_id',
    'other_sample_id',
    'pathology_report_file_name',
    'project_code',
    'somatic_status',
    'tumor_disease_anatomic_site'
    ], axis=1)


In [ ]:
plot_correlation(breast_clean)

In [ ]:
breast_clean.to_csv('breast_clean_TCGA.csv',index=False)

# Chord Diagram

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data

In [ ]:
breast_clean = breast_clean.replace('<', 'less than ', regex=True)
breast_clean = breast_clean.replace('>', 'greater than ', regex=True)
breast_clean=breast_clean.applymap(str)

In [ ]:
breast_clean.head(3)

In [ ]:
hv.Chord(breast_clean)

In [ ]:
chord = hv.Chord((breast_clean,)).select(value=(5, None))
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), 
               labels='name', node_color=dim('index').str()))